In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset
import torchvision.transforms as transforms
from torchvision import datasets
from torch.autograd import Variable
import torch.nn.functional as F
import os

normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                    std=[x/255.0 for x in [63.0, 62.1, 66.7]])  #change pixel intensity range

transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.1307,), (0.3081,))])

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transform
)

100%|██████████| 26421880/26421880 [00:00<00:00, 118583518.33it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 4970284.76it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 59947317.56it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10763846.96it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [ ]:
from torch.utils.data import random_split

val_size = len(test_data) // 2
test_size = len(test_data) - val_size
test_data, val_data = random_split(test_data, [test_size, val_size])

len(val_data), len(test_data)

(5000, 5000)

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00


In [ ]:
import wandb

hyperparameter_defaults = dict(
    dropout = 0.5,
    channels_one = 16,
    channels_two = 32,
    batch_size = 100,
    learning_rate = 0.001,
    epochs = 2,
    )

wandb.init(config=hyperparameter_defaults, project="pytorch-wandb-tutorial", name = "fashion-mnist")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:

# access all HPs through wandb.config
config = wandb.config

#create model class
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()

        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=config.channels_one, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)

        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=config.channels_one, out_channels=config.channels_two, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        self.dropout = nn.Dropout(p=config.dropout)

        # Fully connected 1 (readout)
        self.fc1 = nn.Linear(config.channels_two*4*4, 10)



    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)

        # Max pool 1
        out = self.maxpool1(out)

        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)

        # Max pool 2
        out = self.maxpool2(out)

        # Resize
        # Original size: (100, 32, 7, 7)
        # out.size(0): 100
        # New out size: (100, 32*7*7)
        out = out.view(out.size(0), -1)
        out = self.dropout(out)
        # Linear function (readout)
        out = self.fc1(out)

        return out


In [ ]:
label_names = [
  "T-shirt or top",
  "Trouser",
  "Pullover",
  "Dress",
  "Coat",
  "Sandal",
  "Shirt",
  "Sneaker",
  "Bag",
  "Boot"]

train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                          batch_size=config.batch_size,
                                          shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_data,
                                        batch_size=config.batch_size,
                                        shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                        batch_size=config.batch_size,
                                        shuffle=False)

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = CNNModel()
model = model.to(device)
#track gradients
# wandb.watch(model)
#loss
criterion = nn.CrossEntropyLoss()
# Tell wandb to watch what the model gets up to: gradients, weights, and more!
wandb.watch(model, criterion, log="all", log_freq=10)

#optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

print(model)

CNNModel(
  (cnn1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (cnn2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
!pip install prettytable

In [ ]:
from prettytable import PrettyTable
def model_parameters_table(model):
    table = PrettyTable(["Layer", "Parameter Count"])

    total_params = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_params = param.numel()
            total_params += num_params
            table.add_row([name, num_params])

    table.add_row(["Total", total_params])
    return table

In [ ]:
print(model_parameters_table(model))

+-------------+-----------------+
|    Layer    | Parameter Count |
+-------------+-----------------+
| cnn1.weight |       400       |
|  cnn1.bias  |        16       |
| cnn2.weight |      12800      |
|  cnn2.bias  |        32       |
|  fc1.weight |       5120      |
|   fc1.bias  |        10       |
|    Total    |      18378      |
+-------------+-----------------+


In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 50.9 MB/s eta 0:00:00


In [ ]:

def Train_model(model):
  iter = 0
  total_loss_train = []
  total_loss_validation = []
  # tell wandb to get started
  for epoch in range(config.epochs):
    for i, (images, labels) in enumerate(train_loader):

        batch_loss_train=0
        batch_loss_val=0
        num_batches = 0
        images, labels= images.to(device), labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        batch_loss_train+=loss.item()

        iter += 1
        num_batches+=1

        if iter % 100 == 0:
            # Calculate Accuracy
            correct = 0.0
            correct_arr = [0.0] * 10
            total = 0.0
            total_arr = [0.0] * 10

            # Iterate through test dataset
            for images, labels in test_loader:
                images = Variable(images)
                images, labels = images.to(device), labels.to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)
                correct += (predicted == labels).sum()
                loss_test = criterion(outputs, labels)
                for label in range(10):
                    correct_arr[label] += (((predicted == labels) & (labels==label)).sum())
                    total_arr[label] += (labels == label).sum()

            accuracy = correct / total

            metrics = {'accuracy': accuracy, 'loss_train': loss, 'loss_test': loss_test}
            for label in range(10):
                metrics['Accuracy ' + label_names[label]] = correct_arr[label] / total_arr[label]   #tells wand to log the metrics defined earlier
            wandb.log(metrics)

            # Print Loss
            print('Iteration: {0} Loss_train: {1:.2f} Loss_test: {1:.2f} Accuracy: {2:.2f}'.format(iter, loss,loss_test, accuracy))

    batch_loss_train/=num_batches
    total_loss_train.append(batch_loss_train)

    correct = 0
    total = 0
    count = 0
    with torch.no_grad():

      for data in val_loader:
          # print("s")
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          total += labels.size(0)
          loss_val = criterion(outputs, labels)

          batch_loss_val += loss_val.item()
          count+= 1

      total_loss_validation += [batch_loss_val/count]
      print("Validation loss is ",batch_loss_val/count)

  torch.save(model.state_dict(), os.path.join(wandb.run.dir, "model.pt"))
  path_onnx="model.pt"
  dummy_input = torch.randn(4, 1, 28, 28, device=device)
  torch.onnx.export(model, dummy_input, path_onnx)
  wandb.save(path_onnx)
  wandb.save(path_onnx)
  wandb.save(os.path.join(os.path.join(wandb.run.dir, "model.pth"), "scratch nodel", "saved_model.pth"))
  wandb.finish()



In [ ]:
Train_model(model)

Iteration: 100 Loss_train: 0.79 Loss_test: 0.79 Accuracy: 0.77
Iteration: 200 Loss_train: 0.72 Loss_test: 0.72 Accuracy: 0.68
Iteration: 300 Loss_train: 0.66 Loss_test: 0.66 Accuracy: 0.63
Iteration: 400 Loss_train: 0.58 Loss_test: 0.58 Accuracy: 0.59
Iteration: 500 Loss_train: 0.45 Loss_test: 0.45 Accuracy: 0.55
Iteration: 600 Loss_train: 0.61 Loss_test: 0.61 Accuracy: 0.51
Validation loss is  0.47454928278923036
Iteration: 700 Loss_train: 0.60 Loss_test: 0.60 Accuracy: 0.50
Iteration: 800 Loss_train: 0.46 Loss_test: 0.46 Accuracy: 0.53
Iteration: 900 Loss_train: 0.41 Loss_test: 0.41 Accuracy: 0.53
Iteration: 1000 Loss_train: 0.37 Loss_test: 0.37 Accuracy: 0.57
Iteration: 1100 Loss_train: 0.40 Loss_test: 0.40 Accuracy: 0.47
Iteration: 1200 Loss_train: 0.37 Loss_test: 0.37 Accuracy: 0.49
Validation loss is  0.43461878061294557


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Accuracy Bag,▂▁▇▅█▇▆█▇▆█▇
Accuracy Boot,▃▁▄▆▇▆▅▇██▇▇
Accuracy Coat,▁▄▆▄▄▄▄▄▄▅▄█
Accuracy Dress,▁▅▅█▇▇▇▆▅▇▇█
Accuracy Pullover,▅▅▁▄▆▆▇▆██▆▅
Accuracy Sandal,▁▆▆▆▇▆▇▆█▆█▇
Accuracy Shirt,▁▂▃▆▅▆▆█▆▅▇▆
Accuracy Sneaker,▃▂▄▆▁▇▇█▆▆▇█
Accuracy T-shirt or top,▃▄▆▁▆▅▃▄▇█▇▆
Accuracy Trouser,▁▃▄▆▅▆▅▇█▇▆█
accuracy,▁▃▄▆▆▆▇▇▇▇██


In [ ]:

normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]],
                                    std=[x/255.0 for x in [63.0, 62.1, 66.7]])  #change pixel intensity range

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize(96),
                              transforms.Normalize((0.1307,), (0.3081,))])

training_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)

test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)


100%|██████████| 170498071/170498071 [00:05<00:00, 28836073.04it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [ ]:

wandb.init(config=hyperparameter_defaults, project="pytorch-wandb-tutorial", name = "pretrained model")

wandb: Currently logged in as: saurabhyd. Use `wandb login --relogin` to force relogin


In [ ]:
from torch.utils.data import random_split

val_size = len(test_data) // 2
test_size = len(test_data) - val_size
test_data, val_data = random_split(test_data, [test_size, val_size])

len(val_data), len(test_data)

(5000, 5000)

In [ ]:
label_names =["Airplane","Automobile"
,"Bird"
,"Cat"
,"Deer"
,"Dog"
,"Frog"
,"Horse"
,"Ship"
,"Truck"]

train_loader = torch.utils.data.DataLoader(dataset=training_data,
                                          batch_size=config.batch_size,
                                          shuffle=True)

val_loader = torch.utils.data.DataLoader(dataset=val_data,
                                        batch_size=config.batch_size,
                                        shuffle=False)

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                        batch_size=config.batch_size,
                                        shuffle=False)

In [ ]:
import torchvision
pretrained_model = torchvision.models.alexnet(weights='DEFAULT').to(device)

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 127MB/s]


In [ ]:
print(pretrained_model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [ ]:
print(model_parameters_table(pretrained_model))

+---------------------+-----------------+
|        Layer        | Parameter Count |
+---------------------+-----------------+
|  features.0.weight  |      23232      |
|   features.0.bias   |        64       |
|  features.3.weight  |      307200     |
|   features.3.bias   |       192       |
|  features.6.weight  |      663552     |
|   features.6.bias   |       384       |
|  features.8.weight  |      884736     |
|   features.8.bias   |       256       |
|  features.10.weight |      589824     |
|   features.10.bias  |       256       |
| classifier.1.weight |     37748736    |
|  classifier.1.bias  |       4096      |
| classifier.4.weight |     16777216    |
|  classifier.4.bias  |       4096      |
| classifier.6.weight |     4096000     |
|  classifier.6.bias  |       1000      |
|        Total        |     61100840    |
+---------------------+-----------------+


In [ ]:
pretrained_model.classifier = nn.Sequential(
    nn.Linear(in_features=9216, out_features=4096),
    nn.ReLU(),
    nn.Linear(in_features=4096, out_features=10)
)

print(pretrained_model)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=4096, bias=True)
    (1): ReLU()
    (2): Linear(in_featu

In [ ]:
print(model_parameters_table(pretrained_model))


+---------------------+-----------------+
|        Layer        | Parameter Count |
+---------------------+-----------------+
|  features.0.weight  |      23232      |
|   features.0.bias   |        64       |
|  features.3.weight  |      307200     |
|   features.3.bias   |       192       |
|  features.6.weight  |      663552     |
|   features.6.bias   |       384       |
|  features.8.weight  |      884736     |
|   features.8.bias   |       256       |
|  features.10.weight |      589824     |
|   features.10.bias  |       256       |
| classifier.0.weight |     37748736    |
|  classifier.0.bias  |       4096      |
| classifier.2.weight |      40960      |
|  classifier.2.bias  |        10       |
|        Total        |     40263498    |
+---------------------+-----------------+


In [ ]:
for name, params in pretrained_model.features.named_parameters():
  params.requires_grad = False

print(model_parameters_table(pretrained_model))

+---------------------+-----------------+
|        Layer        | Parameter Count |
+---------------------+-----------------+
| classifier.0.weight |     37748736    |
|  classifier.0.bias  |       4096      |
| classifier.2.weight |      40960      |
|  classifier.2.bias  |        10       |
|        Total        |     37793802    |
+---------------------+-----------------+


In [ ]:
def Train_model(model):
  iter = 0
  total_loss_train = []
  total_loss_validation = []

  for epoch in range(config.epochs):
    for i, (images, labels) in enumerate(train_loader):

        batch_loss_train=0
        batch_loss_val=0
        num_batches = 0
        images, labels= images.to(device), labels.to(device)

        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()

        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()

        batch_loss_train+=loss.item()

        iter += 1
        num_batches+=1

        if iter % 100 == 0:
            # Calculate Accuracy
            correct = 0.0
            correct_arr = [0.0] * 10
            total = 0.0
            total_arr = [0.0] * 10

            # Iterate through test dataset
            for images, labels in test_loader:
                images = Variable(images)
                images, labels = images.to(device), labels.to(device)

                # Forward pass only to get logits/output
                outputs = model(images)

                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)

                # Total number of labels
                total += labels.size(0)
                correct += (predicted == labels).sum()
                loss_test = criterion(outputs, labels)
                for label in range(10):
                    correct_arr[label] += (((predicted == labels) & (labels==label)).sum())
                    total_arr[label] += (labels == label).sum()

            accuracy = correct / total

            metrics = {'accuracy': accuracy, 'loss_train': loss, 'loss_test': loss_test}
            for label in range(10):
                metrics['Accuracy ' + label_names[label]] = correct_arr[label] / total_arr[label]   #tells wand to log the metrics defined earlier
            wandb.log(metrics)

            # Print Loss
            print('Iteration: {0} Loss_train: {1:.2f} Loss_test: {1:.2f} Accuracy: {2:.2f}'.format(iter, loss,loss_test, accuracy))

    batch_loss_train/=num_batches
    total_loss_train.append(batch_loss_train)

    correct = 0
    total = 0
    count = 0
    with torch.no_grad():

      for data in val_loader:
          images, labels = data
          images, labels = images.to(device), labels.to(device)
          outputs = model(images)
          total += labels.size(0)
          loss_val = criterion(outputs, labels)

          batch_loss_val += loss_val.item()
          count+= 1

      total_loss_validation += [batch_loss_val/count]
      print("Validation loss is ",batch_loss_val/count)

  torch.save(model.state_dict(), os.path.join(wandb.run.dir, "model.pt"))
  # path_onnx="model.pt"
  # dummy_input = torch.randn(4, 3, 96, 96, device=device)
  # torch.onnx.export(model, dummy_input, path_onnx)
  # wandb.save(path_onnx)
  wandb.save(os.path.join(os.path.join(wandb.run.dir, "model.pth"), "pretrained model", "saved_model.pth"))
  wandb.finish()




In [ ]:
Train_model(pretrained_model)

Error: You must call wandb.init() before wandb.log()

In [ ]:
wandb.finish()

Accuracy Airplane,▁▁▁▁
Accuracy Automobile,▁▁▁▁
Accuracy Bird,▁▁▁▁
Accuracy Cat,▁▁▁▁
Accuracy Deer,▁▁▁▁
Accuracy Dog,▁▁▁▁
Accuracy Frog,▁▁▁▁
Accuracy Horse,▁▁▁▁
Accuracy Ship,▁▁▁▁
Accuracy Truck,▁▁▁▁
accuracy,▁▁▁▁
